In [192]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
import re
import numpy as np
import statistics
import math

In [193]:
recipes_in = pd.read_parquet('../01_DataCleansing/recipes_in.parquet')
users_in = pd.read_csv('../01_DataCleansing/users_in.csv')

### Only Keep Recipes that Have Values in Techniques Column in This Test

In [194]:
def check_0(x):
    if sum(x.values())!=0:
        return 'Y'
    else:
        return 'N'

In [195]:
recipes=recipes_in.copy()
recipes['check']=recipes.apply(lambda x:check_0(x['techniques']), axis=1)
recipes=recipes[recipes['check']=='Y']

In [196]:
tech_mat = np.array(recipes['techniques'].apply(lambda x: list(x.values())).tolist())

### Set Test Users As Those Who Rate At Least 500 Recipes as Non Zero

In [197]:
users_in_exclude0=users_in.copy()
users_in_exclude0=users_in_exclude0[users_in_exclude0['ratings']!=0]
users=list(set(users_in_exclude0['user_id']))
len(users)


24961

In [198]:
user_recipe_count=users_in_exclude0.groupby(['user_id']).count()
test_users=user_recipe_count[user_recipe_count['recipe_id']>=500].reset_index()
test_users=list(test_users['user_id'])
len(test_users)

193

In [199]:
yes=[]
yes_low=[]
for u in test_users:
    input_user_id=u
    idx = np.where((users_in['user_id']==input_user_id) & (users_in['ratings']>=4))
    live_user_recipes=users_in.copy()
    live_user_recipes=live_user_recipes.loc[idx][['user_id','recipe_id']]    
    
    #Calculate similarity scores for all recipes given this user's recipe preferences
    user_techniq = np.sum(tech_mat[recipes.index.isin(live_user_recipes['recipe_id']),:],axis=0)
    dotp = np.sum(tech_mat * user_techniq,axis=1) # Numerator of cosine similarity
    denom = np.linalg.norm(tech_mat,axis=1) * np.linalg.norm(user_techniq) # Denominator of cosine similarity
    a = np.divide(dotp,denom,out=np.zeros_like(denom), where=denom!=0) # Cosine similarity, places where denom is zero are set to zero
    techniq_sim = pd.DataFrame(a,index=recipes.index,columns=['technique_sim']).reset_index()
    tech_rank=techniq_sim.sort_values(by='technique_sim',ascending=False)
    
    #Get top 10 frequently used techniques by the user
    user_pref=pd.merge(live_user_recipes,recipes,how="inner",on='recipe_id')
    
    u_tech=user_pref['techniques'][1:]
    dd=user_pref['techniques'][0]
    for d in u_tech: 
        for key, value in d.items():
            dd[key] += value
    dd=sorted(dd.items(), key=lambda kv: kv[1],reverse=True)
    u=[x[0] for x in dd[0:10]]
    
    #Get top 10 frequently appeared techniques from the top 20 recipes ranked based on calculated similarity score
    high_rank=tech_rank[0:20]
    high_tech=pd.merge(high_rank,recipes,how="inner",on='recipe_id')
    h_tech=high_tech['techniques'][1:]
    dd_high=high_tech['techniques'][0]
    for d in h_tech: 
        for key, value in d.items():
            dd_high[key] += value
    dd_high=sorted(dd_high.items(), key=lambda kv: kv[1],reverse=True)
    h=[x[0] for x in dd_high[0:10]]
    
    #Get top 10 frequently appeared techniques from the bottom 20 recipes ranked based on calculated similarity score
    low_rank=tech_rank[-21:-1]
    low_tech=pd.merge(low_rank,recipes,how="inner",on='recipe_id')
    l_tech=low_tech['techniques'][1:]
    dd_low=low_tech['techniques'][0]
    for d in l_tech: 
        for key, value in d.items():
            dd_low[key] += value
    dd_low=sorted(dd_low.items(), key=lambda kv: kv[1],reverse=True)
    l=[x[0] for x in dd_low[0:10]]
    
    #Check how many of the top 10 frequently appeared techniques from the top 20 recipes ranked based on calculated similarity score also appear in the top 10 frequently used techniques by the user
    y=0
    for i in h:
        if i in u:
            y+=1
    
    #Check how many of the top 10 frequently appeared techniques from the bottowm 20 recipes ranked based on calculated similarity score also appear in the top 10 frequently used techniques by the user
    y_l=0
    for i in l:
        if i in u:
            y_l+=1
    
    yes.append(y)
    yes_low.append(y_l)
    

In [200]:
import collections


collections.Counter(yes)

Counter({10: 161, 9: 28, 0: 1, 6: 1, 1: 2})

In [201]:
collections.Counter(yes_low)

Counter({2: 2, 3: 6, 1: 8, 0: 174, 10: 1, 9: 2})

189 out of 193 users have over 90% of the top 10 frequently appeared techniques from the top 20 recipes ranked based on calculated similarity score that also appear in the top 10 frequently used techniques by the user;

174 out of 193 users have 0% of the top 10 frequently appeared techniques from the bottom 20 recipes ranked based on calculated similarity score that also appear in the top 10 frequently used techniques by the user;